In [135]:
import os, csv, re, json, sys
from math import sqrt, exp, cos, radians
import numpy as np
from os.path import join as pJoin
from datetime import timedelta, datetime
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote
import requests
from dateutil.parser import parse as parse_dt
from omf import feeder
import platform
import pandas as pd
from tempfile import mkdtemp
import pysolar
import pytz
import xml.etree.ElementTree as ET
import xmltodict
from tensorflow import keras


In [55]:
_key = '31dac4830187f562147a946529516a8d'

In [56]:
def pullUscrn(year, station, datatype):
	'''Returns hourly weather data from NOAA's quality-controlled USCRN dataset as array.
	* Documentation: https://www1.ncdc.noaa.gov/pub/data/uscrn/products/hourly02/README.txt
	* List of available stations: https://www1.ncdc.noaa.gov/pub/data/uscrn/products/hourly02'''
	url = ('https://www1.ncdc.noaa.gov/pub/data/uscrn/products/hourly02/{0}/'
		'CRNH0203-{0}-{1}.txt'.format(year, station))
	r = requests.get(url)
	assert r.status_code != 404, "Dataset URL does not exist. " + url 
	# Columns name and index (subtracted 1 from readme's Field #)
	datatypeDict = {
		"T_CALC": 8,
		"T_HR_AVG": 9,
		"T_MAX": 10,
		"T_MIN": 11,
		"P_CALC": 12,
		"SOLARAD": 13,
		"SOLARAD_MAX": 15,
		"SOLARAD_MIN": 17,
		"SUR_TEMP": 20,
		"SUR_TEMP_MAX": 22,
		"SUR_TEMP_MIN": 24,
		"RH_HR_AVG": 26,
		"SOIL_MOISTURE_5": 28,
		"SOIL_MOISTURE_10": 29,
		"SOIL_MOISTURE_20": 30,
		"SOIL_MOISTURE_50": 31,
		"SOIL_MOISTURE_100": 32,
		"SOIL_TEMP_5": 33,
		"SOIL_TEMP_10": 34,
		"SOIL_TEMP_20": 35,
		"SOIL_TEMP_50": 36,
		"SOIL_TEMP_100": 37
		}
	#IF datatype is "irradiance estimate", then run the diffuse/direct_seperator method 
	assert datatype in datatypeDict, "This datatype isn't listed in options!"
	datatypeID = datatypeDict[datatype]
	rawData = [float(x.split()[datatypeID]) for x in r.text.splitlines() if len(x) != 0]
	if datatype == "IRRADIENCE_DIFFUSE":
		#Now get diffuse component, write in place in data array
		diffuse = list(map(get_diffuse_solar_component, rawData))
		# #Subtract diffuse from raw to get direct component
		direct = list(map(lambda x, y: x-y,rawData, diffuse))
		direct_diffuse = list(zip(direct, diffuse))
		raw_diffuse = list(zip(rawData, diffuse))
		return raw_diffuse #direct_diffuse
	return rawData

In [67]:
#Standard positional arguments are for TX_Austin
def _getDarkSkyCloudCoverForYear(year='2018', lat=30.581736, lon=-98.024098, key=_key, units='si'):
	cloudCoverByHour = {}
	pressureByHour = {}
	coords = '%0.2f,%0.2f' % (lat, lon)
	times = list(pd.date_range('{}-01-01'.format(year), '{}-12-31'.format(year), freq='D'))
	while times:
		time = times.pop(0)
		print(time)
		url = 'https://api.darksky.net/forecast/%s/%s,%s?exclude=daily,alerts,minutely,currently&units=%s' % (key, coords, time.isoformat(), units ) 
		res = requests.get(url).json()
		try:
			dayData = res['hourly']['data']
		except KeyError:
			print("No day data!!!!!!")
			continue
		for hour in dayData:
			try:
				cloudCoverByHour[hour['time']] = hour['cloudCover']
				#Darksky result in hpascals, model trained in mbar. 1 - 1 transformation.
				pressureByHour[hour['time']] = hour['pressure']
			except KeyError:
				print("No Cloud Cover Data")
				pass
	return cloudCoverByHour, pressureByHour

In [76]:
def _getUscrnData(year='2018', location='TX_Austin_33_NW', dataType="SOLARAD"):
    ghiData = pullUscrn(year, location, dataType)
    return ghiData

In [69]:
def getSolarZenith(lat, lon, datetime, timezone):
    date = pytz.timezone(timezone).localize(datetime)
    solar_altitude = pysolar.solar.get_altitude(lat,lon,date)
    solar_zenith = 90 - solar_altitude
    return solar_zenith

In [70]:
def preparePredictionVectors(year='2018', lat=30.581736, lon=-98.024098, station='TX_Austin_33_NW', timezone='US/Central'):
    cloudCoverData, pressureData = _getDarkSkyCloudCoverForYear(year, lat, lon)
    ghiData = _getUscrnData(year, station, dataType="SOLARAD")
    #for each 8760 hourly time slots, make a timestamp for each slot, look up cloud cover by that slot
    #then append cloud cover and GHI reading together
    start_time = datetime(int(year),1,1,0)
    cosArray = []
    input_array = []
    for i in range(len(ghiData)): #Because ghiData is leneth 8760, one for each hour of a year
        time = start_time + timedelta(minutes=60*i)
        tstamp = int(datetime.timestamp(time))
        hour = time.hour
        minute = time.minute
        try:
            cloudCover = cloudCoverData[tstamp]
            pressure = pressureData[tstamp]
        except KeyError:
            cloudCover = 0
            pressure = 0
        #I have my cloud cover, iterate over my ghi and cosine arrays
        solar_zenith = getSolarZenith(lat, lon, time, timezone)
        #Get cosine of solar zenith, this is going to be used later in dni calculation. Make sure its in radians.
        cosOfSolarZenith = cos(solar_zenith*0.0175)
        ghi = ghiData[i]
        input_array.append([ghi, cloudCover, hour, minute, solar_zenith, pressure])
        cosArray.append(cosOfSolarZenith)
    return input_array, ghiData, cosArray

In [71]:
# def combine_training_arrays(df, input_arry):
#     ghi  = df['GHI'].values
#     cloud_cover = df['Cloud Cover'].values
#     hours = df['Hour'].values
#     minutes = df['Minute'].values
#     solar_zenith = df['Solar Zenith Angle'].values
#     pressure = df['Pressure'].values
#     #Sanity check, should all be 8760
# #     assert len(pressure)==len(solar_zenith)==len(minutes)==len(hours)==len(cloud_cover)==len(ghi)==8760, "len of input array not 8760"
#     ar = np.array([ghi, cloud_cover, hours, minutes, solar_zenith, pressure]).T
#     input_arry = np.concatenate((input_arry, ar))
#     output_arr = np.concatenate((output_arr, np.array(dhi).T))
#     return input_arry


In [72]:
def predictNeuralNet(input_array, model_path):
    model = keras.models.load_model(model_path)
    #Takes in numpy array of proper shape
    """
    Ghi
    Cloud Cover 
    Hours
    Minutes
    Solar Zenith
    Pressure
    DHI"""
    preds = model.predict(input_array)
    return preds

In [132]:
def run_estimation(uscrn_station=[30.621,-98.08,'US/Central']):
    year='2018'
    lat = uscrn_station[0]
    lon = uscrn_station[1]
    timezone = uscrn_station[2]
    input_array, ghi_array, cos_array = preparePredictionVectors()
    print("input array created")
    dhi_preds = list(predictNeuralNet(input_array, 'Neural_Nets/Neural_Net_National'))
    dhi_preds = [float(i) for i in dhi_preds]
    print("preds made")
    dniXCosTheta = [ghi_array[i] - dhi_preds[i] for i in range(0, len(ghi_array))] #This is cos(theta) * DNI
    print(len(dniXCosTheta))
    print("cos theta calculation made")
    dni_array = ([dniXCosTheta[i]/cos_array[i] for i in range(len(dniXCosTheta))])
    print(len(dni_array))
    print(type(dhi_preds), type(dni_array), type(ghi_array))
    result = list(zip((dhi_preds), (ghi_array), (dni_array)))
    assert len(result) == len(input_array)
    print("result is ", type(result), len(result))
    print(result)
    return result

In [133]:
def run_tests():
    run_estimation()
    

In [134]:
run_tests()

2018-01-01 00:00:00
2018-01-02 00:00:00
2018-01-03 00:00:00
2018-01-04 00:00:00
2018-01-05 00:00:00
2018-01-06 00:00:00
2018-01-07 00:00:00
2018-01-08 00:00:00
2018-01-09 00:00:00
2018-01-10 00:00:00
2018-01-11 00:00:00
2018-01-12 00:00:00
2018-01-13 00:00:00
2018-01-14 00:00:00
2018-01-15 00:00:00
2018-01-16 00:00:00
2018-01-17 00:00:00
2018-01-18 00:00:00
2018-01-19 00:00:00
2018-01-20 00:00:00
2018-01-21 00:00:00
2018-01-22 00:00:00
2018-01-23 00:00:00
2018-01-24 00:00:00
2018-01-25 00:00:00
2018-01-26 00:00:00
2018-01-27 00:00:00
2018-01-28 00:00:00
2018-01-29 00:00:00
2018-01-30 00:00:00
No Cloud Cover Data
2018-01-31 00:00:00
2018-02-01 00:00:00
2018-02-02 00:00:00
2018-02-03 00:00:00
2018-02-04 00:00:00
2018-02-05 00:00:00
2018-02-06 00:00:00
2018-02-07 00:00:00
2018-02-08 00:00:00
2018-02-09 00:00:00
2018-02-10 00:00:00
2018-02-11 00:00:00
2018-02-12 00:00:00
2018-02-13 00:00:00
2018-02-14 00:00:00
2018-02-15 00:00:00
2018-02-16 00:00:00
2018-02-17 00:00:00
2018-02-18 00:00:00


2018-11-30 00:00:00
2018-12-01 00:00:00
2018-12-02 00:00:00
2018-12-03 00:00:00
2018-12-04 00:00:00
2018-12-05 00:00:00
2018-12-06 00:00:00
2018-12-07 00:00:00
2018-12-08 00:00:00
2018-12-09 00:00:00
2018-12-10 00:00:00
2018-12-11 00:00:00
2018-12-12 00:00:00
2018-12-13 00:00:00
2018-12-14 00:00:00
2018-12-15 00:00:00
2018-12-16 00:00:00
2018-12-17 00:00:00
2018-12-18 00:00:00
2018-12-19 00:00:00
2018-12-20 00:00:00
2018-12-21 00:00:00
2018-12-22 00:00:00
2018-12-23 00:00:00
2018-12-24 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
input array created
preds made
8760
cos theta calculation made
8760
<class 'list'> <class 'list'> <class 'list'>
result is  <class 'list'> 8760
[(0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, -0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.